In [5]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'jimmy@programmingblockchain.com secret passphrase'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
print(my_address)
target_amount = 10000000
fee = 6000

response = requests.get('https://blockstream.info/testnet/api/address/{}/utxo'.format(my_address))
utxos = response.json()
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

student_addresses = '''mgHULz6rZsc9J9PbMVt4os2UGBCZV8YCHm
mpB21Y43biNhwXx22LXZkNFt1BWKFZQujr
mucet8M9rRDHa4jhuAH6ckYrKHHETpfwUF
mkMNZ1wpPV8iX29RCc8pq4tGywb1B45HKJ
mpFo5MGsFmqMmPGwUsXcm8cefrC44pLWR6
n4beeevKU63Py8YhCRWD5MHSGcJgNSNtbR'''
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = target_amount
    output_amount += target_satoshis
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_satoshis,
        script_pubkey=script_pubkey,
    ))

change_satoshis = total - fee - output_amount
h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv)

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv
010000000f1f48f442b69bc35e9fcef76ed5af17c00a7d4e0b62b71c4b68399f0662a00c850f0000006a473044022072b69597fa9b8be15169586aa7a565d391edf0db757ecb3aaa32f38ffa2629330220416bfd12c1da001412ecfdd73ec1aef038f2c5e1cbdf03373cbe484523ed599e0121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6effffffff0641e02bb2176c98f04ff3de51de71bf13a4008ba8d3a1ef14aec795f8a11acd000000006b483045022100b605313b57257e83447309850ea4bb9b8c0fe35a41b4e7af921f917eee7a65fc022070ae8b03eddb7dac537e9b2f4d80ea7fd838c8fc4d3efbce560213f9d0079e8e0121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6effffffffb4709fa774fb58800fc01fb1c4dbbaaa12d157eb3aa462af9592a0971a600235000000006a47304402205ecfcd6fd6ac22ebbaf0b34496028a113a3c3e5edb501e2693e8fdd70512027102202ed2fffe4ea5ffd54afd8ac76a09400c879812d5fc1cb65713e8a851b99b84810121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6effffffffb318881e5488ac034101e82f4766b8e7b5c0df25f351b70c3736a192695644780000000

In [1]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'jimmy@programmingblockchain.com secret passphrase'

secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
fee = 6000

response = requests.get('https://blockstream.info/testnet/api/address/{}/utxo'.format(my_address))
utxos = response.json()
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

student_addresses = '''mgHULz6rZsc9J9PbMVt4os2UGBCZV8YCHm
mpB21Y43biNhwXx22LXZkNFt1BWKFZQujr
mucet8M9rRDHa4jhuAH6ckYrKHHETpfwUF
mkMNZ1wpPV8iX29RCc8pq4tGywb1B45HKJ
mpFo5MGsFmqMmPGwUsXcm8cefrC44pLWR6
n4beeevKU63Py8YhCRWD5MHSGcJgNSNtbR'''
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))

# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = randint(5000000, 20000000)
    output_amount += target_satoshis
    # create a new tx output for target
    tx_outs.append(TxOut(target_satoshis, script_pubkey))

change_satoshis = total - fee - output_amount

h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv)

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 500000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

0100000008b5c927b46f55b95f6a7ca8ba26f5297bac458292cd738d8aacbf3479db36435d010000006b483045022100858a724e4b0442e09b193f076f362deed3ae1141aa05143fa8694b6577b6002a02205c743ec1e4707cac85f9cd582117c1e3655d8d188007346ca9ba739f6e3077f50121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6effffffffe605adc224d883ea809af07ba39db8f11654381bce04409c2d9101f4488644b7000000006b483045022100f0e58e2a286ed9aa39f01116f125566528b0d2191cc33f880eaabc588433ca1a02207e869944032b6bd9ed7037b619d1e8e0f256a235df7cb8b07e9bb03952dae85c0121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6effffffff613ff2ca26caed688dd280887e7c68a55cfbdf5b7f2b7122a4a10dd1b2bb0fc6060000006b483045022100a797df1ab6d891e2423a2e5dc299f5f04c463dc9d76c643918ee92b93f7b15fa02207b0742c88614c4c47bd3959bf380435dac0b32a6e2ebe95234ecd9018d120fc30121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6effffffffb5c927b46f55b95f6a7ca8ba26f5297bac458292cd738d8aacbf3479db36435d000000006a47304402205169a4974d5f5430dd

In [ ]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'Jimmy Song'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
print(my_address)
fee = 0.001

response = requests.get('https://testnet.blockchain.info/unspent?active={}'.format(my_address))
utxos = response.json()['unspent_outputs']
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

consolidation_address = '''mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv'''
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    if utxo['confirmations'] == 0:
        continue
    if utxo['tx_hash_big_endian'] in (
        '3d553a0343c160f7429f19bab57f0efdf146847f279dcb86a881319e2e21ff4a',
        '869f7d3ec5a910f486a203f52980cd60df4d3d586d738fad47858757406bd04f',
    ):
        continue
    
    prev_tx = bytes.fromhex(utxo['tx_hash_big_endian'])
    prev_index = utxo['tx_output_n']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
# decode the hash160 from the target address
h160 = decode_base58(consolidation_address)
# convert hash160 to p2pkh script
script_pubkey = p2pkh_script(h160)
target_satoshis = total - int(fee*100000000)
print(target_satoshis)
# create a new tx output for target
tx_outs.append(TxOut(
    amount=target_satoshis,
    script_pubkey=script_pubkey,
))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv)

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())